# Yelp API - Lab



## Introduction 

Now that we've seen how the Yelp API works, it's time to put those API and SQL skills to work in order to do some basic business analysis! Taking things a step further, you'll also independently explore how to perform pagination in order to retrieve a full results set from the Yelp API!

## Objectives

You will be able to:
* Create a DB on AWS to store information from Yelp about businesses
* Create HTTP requests to get data from Yelp API
* Parse HTTP responses and insert the information into your DB
* Perform pagination to retrieve troves of data!
* Write SQL queries to answer questions about your data 

## Problem Introduction

For this lab you will analyze the yelp data for a group of businesses to learn more about an industry. You will choose a type of business (Italian Restuarants, Nail Salons, Crossfit gyms) and a location to analyze. Then you will get data from the Yelp API, store that data in a SQL Database on AWS, and write queries to answer questions about the data. 


### Process:

1. Read through the SQL questions and the API documentation to determine which pieces of information you need to pull from the Yelp API.

2. Create a DB schema with 2 tables. One for the businesses and one for the reviews.

3. Create code to:
  - Perform a search of businesses using pagination
  - Parse the API response for specific data points
  - Insert the data into your AWS DB

4. Use the functions above in a loop that will paginate over the results to retrieve all of the results. 

5. Create functions to:
  - Retrieve the reviews data of one business
  - Parse the reviews response for specific review data
  - Insert the review data into the DB

6. Using SQL, query all of the business IDs. Using the 3 Python functions you've created, run your business IDs through a loop to get the reviews for each business and insert them into your DB.

7. Write SQL queries to answer the following questions about your data.


Bonus Steps:  
- Place your helper functions in a package so that your final notebook only has the major steps listed.
- Rewrite your business search functions to be able take an argument for the type of business you are searching for.
- Add another group of businesses to your database.


 
## SQL Questions:

- Which are the 5 most reviewed businesses?
- What is the highest rating recieved in your data set and how many businesses have that rating?
- What percentage of businesses have a rating greater than or  4.5?
- What percentage of businesses have a rating less than 3?
- What is the average rating of restaurants that have a price label of one dollar sign? Two dollar signs? Three dollar signs? 
- Return the text of the reviews for the most reviewed restaurant. 
- Return the name of the business with the most recent review. 
- Find the highest rated business and return text of the most recent review. If multiple business have the same rating, select the restaurant with the most reviews. 
- Find the lowest rated business and return text of the most recent review.  If multiple business have the same rating, select the restaurant with the least reviews. 


## Part 1 - Understanding your data and question

Lok at the question and determine what data you will need to store in your database in order to answer the questions. Start to thin about your DB schema. What tables will you want to create and what columns will you ahve for those tables. 

Look at the API documentation, and determine what fields of the API response you will match up with the columns you want in your DB tables. 


https://www.yelp.com/developers/documentation/v3/get_started

## Part 2 - Set up the DB

Now that you are familiar with the data, create your SQL queries to create the DB and the appropriate tables. 

### Foriegn Keys

For this project you will have two tables that you need to link using a foreign key. Below is some information about how to create a foriegn key.  

http://www.mysqltutorial.org/mysql-foreign-key/



## Part 3 - Create ETL pipeline for the business data from the API

Now that you know what data you need from the API, you want to write code that will execute a api call, parse those results and then insert the results into the DB.  

It is helpful to break this up into three different functions (*api call, parse results, and insert into DB*) and then you can write a function/script that pull the other three functions together. 

Let's first do this for the Business endpoint.

In [6]:
import requests
import json
client_id = 'DxQeJSiKJUmWIre2XGaKGg'
api_key = '37K09M8JaffyRQL_0ZZtHQCZxV_HsOlyXzY1wys6XhR6sJOJNf1H3ojayRAQnebOywHxZJOmZGxVvvAECfrX5uPgcunVHC8E6ThWcloYXHreayE6v3K2cbQYg_0pXnYx'

In [7]:
    term = 'pub'
    location = 'NY'
    SEARCH_LIMIT = 10

    url = 'https://api.yelp.com/v3/businesses/search'

    headers = {
        'Authorization': 'Bearer {}'.format(api_key),
    }

    url_params = {
                    'term': term.replace(' ', '+'),
                    'location': location.replace(' ', '+'),
                    'limit': SEARCH_LIMIT
            }
    response = requests.get(url, headers=headers, params=url_params)
print(response)
print(type(response.text))
print(response.text)

<Response [200]>
<class 'str'>
{"businesses": [{"id": "oAjwUCzyvH9gBIOOlf8PtQ", "alias": "the-stinking-rose-pub-and-grindhouse-harcourt", "name": "The Stinking Rose Pub & Grindhouse", "image_url": "https://s3-media4.fl.yelpcdn.com/bphoto/xsYj7Vzs5j_SHi994_9I7g/o.jpg", "is_closed": false, "url": "https://www.yelp.com/biz/the-stinking-rose-pub-and-grindhouse-harcourt?adjust_creative=DxQeJSiKJUmWIre2XGaKGg&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=DxQeJSiKJUmWIre2XGaKGg", "review_count": 1, "categories": [{"alias": "pubs", "title": "Pubs"}], "rating": 5.0, "coordinates": {"latitude": 44.30876, "longitude": -77.80071}, "transactions": [], "price": "$$", "location": {"address1": "26 Bridge West", "address2": "", "address3": "", "city": "Harcourt", "zip_code": "K0L", "country": "CA", "state": "ON", "display_address": ["26 Bridge West", "Harcourt, ON K0L", "Canada"]}, "phone": "+17056530001", "display_phone": "+1 705-653-0001", "distance": 810.2538134017361}, {"id"

In [8]:
response.json().keys()

dict_keys(['businesses', 'total', 'region'])

In [9]:
for key in response.json().keys():
    print(key)
    value = response.json()[key] #Use standard dictionary formatting
    print(type(value)) #What type is it?
    print('\n') #Seperate out data

businesses
<class 'list'>


total
<class 'int'>


region
<class 'dict'>




In [10]:
import pandas as pd
yelp_data =response.json()
pub = yelp_data['businesses']

In [11]:
pd.DataFrame(pub).head()

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
0,oAjwUCzyvH9gBIOOlf8PtQ,the-stinking-rose-pub-and-grindhouse-harcourt,The Stinking Rose Pub & Grindhouse,https://s3-media4.fl.yelpcdn.com/bphoto/xsYj7V...,False,https://www.yelp.com/biz/the-stinking-rose-pub...,1,"[{'alias': 'pubs', 'title': 'Pubs'}]",5.0,"{'latitude': 44.30876, 'longitude': -77.80071}",[],$$,"{'address1': '26 Bridge West', 'address2': '',...",+17056530001,+1 705-653-0001,810.253813
1,OxqUWRZFUOz3r6_4bh4mfg,cock-and-bull-new-york,Cock & Bull,https://s3-media2.fl.yelpcdn.com/bphoto/-82MfQ...,False,https://www.yelp.com/biz/cock-and-bull-new-yor...,531,"[{'alias': 'british', 'title': 'British'}, {'a...",4.0,"{'latitude': 40.7560908823673, 'longitude': -7...",[restaurant_reservation],$$,"{'address1': '23 W 45th St', 'address2': None,...",+12128191900,(212) 819-1900,5750.495668
2,3NbhYebjZPjLTWZZyRuhsw,the-winslow-new-york-2,The Winslow,https://s3-media2.fl.yelpcdn.com/bphoto/KqK_JD...,False,https://www.yelp.com/biz/the-winslow-new-york-...,527,"[{'alias': 'pubs', 'title': 'Pubs'}, {'alias':...",4.5,"{'latitude': 40.7327766418457, 'longitude': -7...",[restaurant_reservation],$$,"{'address1': '243 E 14th St', 'address2': '', ...",+12127777717,(212) 777-7717,3132.702360
3,ZnhRUBos9FGQZrzbcPn6Vg,o-haras-restaurant-and-pub-new-york,O'Hara's Restaurant and Pub,https://s3-media2.fl.yelpcdn.com/bphoto/nHPOfx...,False,https://www.yelp.com/biz/o-haras-restaurant-an...,458,"[{'alias': 'irish_pubs', 'title': 'Irish Pub'}...",4.0,"{'latitude': 40.7093473589949, 'longitude': -7...",[],$$,"{'address1': '120 Cedar St', 'address2': 'Ste ...",+12122673032,(212) 267-3032,1575.039638
4,bkIMbvXOcztDVz8bpyhMzQ,the-dead-poet-new-york,The Dead Poet,https://s3-media4.fl.yelpcdn.com/bphoto/fW9WcV...,False,https://www.yelp.com/biz/the-dead-poet-new-yor...,424,"[{'alias': 'bars', 'title': 'Bars'}, {'alias':...",4.0,"{'latitude': 40.78499, 'longitude': -73.97735}",[],$$,"{'address1': '450 Amsterdam Ave', 'address2': ...",+12125955670,(212) 595-5670,8959.249622


In [12]:
import mysql.connector
from mysql.connector import errorcode

In [13]:
cnx = mysql.connector .connect(
    host = "database-1.cl7l8mgndgqj.us-east-2.rds.amazonaws.com",
    user = "KuSin",
    passwd = "Password",
    database = 'yelp'
)
print(cnx)
cursor = cnx.cursor()

ProgrammingError: 1049 (42000): Unknown database 'yelp'

In [14]:
def create_database(cursor, database):
    try:
        cursor.execute(
            "CREATE DATABASE {} DEFAULT CHARACTER SET 'utf8'".format(database))
    except mysql.connector.Error as err:
        print("Failed creating database: {}".format(err))
        exit(1)

In [ ]:
db_name = "yelp"
create_database(cursor, db_name)

In [ ]:
def parsed_tuples(yelp_data):
    for i in yelp_data['businesses']:
        if 'name' in i:
            pass
        else:
            i['name']='nan'
        if 'review_count' in i:
            pass
        else:
            i['review_count']='nan'
        if 'rating' in i:
            pass
        else:
            i['rating']='nan'
        if 'location' in i:
            pass
        else:
            i['location']='nan'
        if 'price' in i:
            pass
        else:
            i['price']='nan'
    parsed_tuple_list = []
    for i in yelp_data['businesses']:
        parsed_tuple_list.append((i['id'], i['name'], i['review_count'], i['rating'], i['location'], i['price']))
    return parsed_tuple_list
aa=parsed_tuples(yelp_data)
aa


In [ ]:
test =  (
    "CREATE TABLE Businesses ("
    "  id varchar(50) PRIMARY KEY,"
    "  name varchar(100),"
    "  price varchar(6),"
    "  rating float(6),"
    "  review_count int(10),"
    ") ENGINE=InnoDB")

In [ ]:
#DB_NAME = 'Pubs'#

# db_name = 'yelp'
TABLES = {}
TABLES['Businesses'] = (
    "CREATE TABLE Businesses ("
    "  id varchar(50) PRIMARY KEY,"
    "  name varchar(100),"
    "  price varchar(6),"
    "  rating float(6),"
    "  review_count int(10)"
    ") ENGINE=InnoDB")

In [ ]:
cursor.execute('USE yelp')

In [ ]:
for table_name in TABLES:
    table_description = TABLES[table_name]
    try:
        print("Creating table {}: ".format(table_name), end='')
        cursor.execute(table_description)
    except mysql.connector.Error as err:
        if err.errno == errorcode.ER_TABLE_EXISTS_ERROR:
            print("already exists.")
        else:
            print(err.msg)
    else:
        print("OK")


In [ ]:
stmt = "INSERT INTO businesses(name, price, rating, review_count) VALUES (%s, %s, %s, %s)".format(name, price, rating, review_count)
cursor.executemany(stmt, aa)
cnx.commit()

In [ ]:
print (TABLES['Businesses'])

In [ ]:
# write a function to parse the API response 
# so that you can easily insert the data in to the DB

In [ ]:
# Write a function to take your parsed data and insert it into the DB

In [ ]:
# Write a script that combines the three functions above into a single process.

## Part 4 -  Create ETL pipeline for the restaurant review data from the API

You've done this for the Businesses, now you need to do this for reviews. You will follow the same process, but your functions will be specific to reviews.

In [ ]:
# write a SQL query to pull back all of the business ids 
# you will need these ids to pull back the reviews for each restaurant

In [ ]:
# write a function that takes a business id 
# and makes a call to the API for reivews


In [ ]:
# Write a function to parse out the relevant information from the reviews

In [ ]:
# write a function to insert the parsed data into the reviews table

In [ ]:
# combine the functions above into a single script  

## Part 5 -  Write SQL queries that will answer the questions posed. 

Now that your data is in the DB, you can answer the questions. Write SQL querires that will answer the questions. **Do not use Pandas in this lab**

In [ ]:
# create connection

In [ ]:
# execute SQL queries

# Extra Reference help

###  Pagination

Returning to the Yelp API, the [documentation](https://www.yelp.com/developers/documentation/v3/business_search) also provides us details regarding the API limits. These often include details about the number of requests a user is allowed to make within a specified time limit and the maximum number of results to be returned. In this case, we are told that any request has a maximum of 50 results per request and defaults to 20. Furthermore, any search will be limited to a total of 1000 results. To retrieve all 1000 of these results, we would have to page through the results piece by piece, retriving 50 at a time. Processes such as these are often refered to as pagination.

Now that you have an initial response, you can examine the contents of the json container. For example, you might start with ```response.json().keys()```. Here, you'll see a key for `'total'`, which tells you the full number of matching results given your query parameters. Write a loop (or ideally a function) which then makes successive API calls using the offset parameter to retrieve all of the results (or 5000 for a particularly large result set) for the original query. As you do this, be mindful of how you store the data. 

**Note: be mindful of the API rate limits. You can only make 5000 requests per day, and APIs can make requests too fast. Start prototyping small before running a loop that could be faulty. You can also use time.sleep(n) to add delays. For more details see https://www.yelp.com/developers/documentation/v3/rate_limiting.**

***Below is sample code that you can use to help you deal with the pagination parameter and bring all of the functions together.***


***Also, something might cause your code to break while it is running. You don't want to constantly repull the same data when this happens, so you should insert the data into the database as you call and parse it, not after you have all of the data***


In [ ]:
def yelp_call(url_params, api_key):
    # your code to make the yelp call
    return data

In [ ]:
def parse_results(results):
    return parsed_results

In [ ]:
def db_insert(cnx, cursor, parsed results):
    # your code to insert and commit the results
    # I would create the connection and cursor outside of this function and then pass it through

In [ ]:
# create a variable  to keep track of which result you are in. 
cur = 0

#set up a while loop to go through and grab the result 
while cur < num and cur < 1000:
    #set the offset parameter to be where you currently are in the results 
    url_params['offset'] = cur
    #make your API call with the new offset number
    results = yelp_call(url_params, api_key)
    
    #after you get your results you can now use your function to parse those results
    parsed_results = parse_results(results)
    
    # use your function to insert your parsed results into the db
    db_insert(parsed_results)
    #increment the counter by 50 to move on to the next results
    cur += 50